# Main Notebook
1. Clean
2. Filter
3. Tokenize

In [ ]:
# !pip install pandarallel
# !pip install nltk
# !pip install spacy
# !python3 -m spacy download en_core_web_md
# nltk.download('punkt')
# !pip install matplotlib

In [1]:
import pandas as pd
import pandarallel
import multiprocessing
import warnings
import re
import string
import itertools

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_colwidth', 5000)
warnings.filterwarnings("ignore")

In [3]:
num_processors = multiprocessing.cpu_count()
workers = num_processors-1
print(f'Using {workers} workers')
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Using 63 workers
INFO: Pandarallel will run on 63 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
import spacy
nlp = spacy.load("en_core_web_md", disable=['parser', 'ner'])

# Data loading and EDA

In [5]:
%%time

df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

CPU times: user 7.86 s, sys: 5.12 s, total: 13 s
Wall time: 17.7 s


(200332, 5)

In [6]:
df_news_final_project.shape

(200332, 5)

In [7]:
df_news_final_project.dtypes

url         object
date        object
language    object
title       object
text        object
dtype: object

In [8]:
# Convert to datetime
df_news_final_project['date'] = pd.to_datetime(df_news_final_project['date'])

In [9]:
df_news_final_project.isnull().sum()

url         0
date        0
language    0
title       0
text        0
dtype: int64

In [10]:
df_news_final_project['url'].value_counts()

url
http://en.people.cn/n3/2021/0318/c90000-9830122.html                                                                            1
https://it-online.co.za/2022/10/19/senior-data-engineer-western-cape-century-city/                                              1
https://goodmenproject.com/featured-content/emerging-standards-for-using-llms-like-chatgpt-in-research-publications/            1
https://hackernoon.com/creating-a-machine-learning-workflow-using-google-bigquery-and-amazon-managed-apache-airflow-c74r31fd    1
https://hackernoon.com/is-your-data-science-team-an-island-61d3f64fd397                                                         1
                                                                                                                               ..
https://sociable.co/technology/how-ai-influenced-habits-and-behavioral-changes-lead-to-financial-freedom-podcast-episode/       1
https://southernmarylandchronicle.com/2023/02/25/using-ai-to-improve-wetland-mapping-a

In [11]:
# Regex pattern to extract website names
pattern = r'(https?://(?:www\.)?\S+\.\w{2,4}(?!html\b)(?:\.\w{2})?\/)'

# Function to extract website names
def extract_website_name(url):
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return ""

# Apply the function to column and create a new column
df_news_final_project['website'] = df_news_final_project['url'].apply(extract_website_name)

In [12]:
df_news_final_project['website'].value_counts()

website
https://www.newsbreak.com/                                                                          3501
https://www.prnewswire.com/                                                                         3221
https://www.einpresswire.com/                                                                       2638
https://venturebeat.com/                                                                            2373
https://www.wfmz.com/                                                                               2213
                                                                                                    ... 
https://www.nnbusinessview.com/                                                                        1
https://www.kildarenow.com/                                                                            1
https://www.moneycontrol.com/news/business/meta-rolls-out-ai-language-model-llama-10157071.html/       1
https://www.dig-in.com/                        

In [13]:
df_news_final_project['language'].value_counts()

language
en    200332
Name: count, dtype: int64

In [14]:
#majority of articles are recently published (in 2023)
df_news_final_project['date'].value_counts()

date
2023-02-07    1237
2023-02-08    1061
2023-04-18    1039
2023-04-25     986
2023-04-27     955
              ... 
2020-05-17      15
2020-12-27      14
2022-02-06      13
2022-01-02      12
2020-09-27      12
Name: count, Length: 1213, dtype: int64

In [15]:
min_date = df_news_final_project['date'].min()
max_date = df_news_final_project['date'].max()

print("Minimum date:", min_date)
print("Maximum date:", max_date)

Minimum date: 2020-01-01 00:00:00
Maximum date: 2023-04-28 00:00:00


In [16]:
df_news_final_project['title'].value_counts()

title
Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice              237
Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice    119
Trump arrested? Putin jailed? Fake AI images spread online                                                                                99
Musk, scientists call for halt to AI race sparked by ChatGPT                                                                              97
Biden administration seeks input on AI safety measures                                                                                    86
                                                                                                                                        ... 
Qualcomm Snapdragon 888 Plus Announced with CPU and AI Upgrades | Beebom                                                                   1
Google 

In [17]:
top_3_titles = df_news_final_project['title'].value_counts().nlargest(3).index
df_news_final_project[df_news_final_project['title'].isin(top_3_titles)]

url   
977                      https://www.wrdw.com/prnewswire/2022/07/15/virtus-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/  \
1469                    https://www.fox19.com/prnewswire/2023/01/19/virtus-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
1538                      https://www.kgns.tv/prnewswire/2023/01/19/virtus-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
1906                                                                       https://www.wiltonbulletin.com/news/politics/article/trump-arrested-putin-jailed-fake-ai-images-17856545.php   
2554          https://www.kjct8.com/prnewswire/2022/01/18/virtus-allianzgi-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
...                                                                                                                                                                                 ...   
196963  https://www.cleveland19.com/prnewswire/2022/05/16/virtus-allianzgi-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
198415         https://www.wlbt.com/prnewswire/2022/01/18/virtus-allianzgi-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
198792            https://www.cleveland19.com/prnewswire/2022/08/18/virtus-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
198964       https://www.kfvs12.com/prnewswire/2022/01/18/virtus-allianzgi-artificial-intelligence-amp-technology-opportunities-fund-discloses-sources-distribution-section-19a-notice/   
199931                                                                                        https://www.kltv.com/2023/03/23/trump-arrested-putin-jailed-fake-ai-images-spread-online/   

             date language   
977    2022-07-15       en  \
1469   2023-01-19       en   
1538   2023-01-19       en   
1906   2023-03-23       en   
2554   2022-01-18       en   
...           ...      ...   
196963 2022-05-16       en   
198415 2022-01-18       en   
198792 2022-08-18       en   
198964 2022-01-18       en   
199931 2023-03-23       en   

                                                                                                                                        title   
977               Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice  \
1469              Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
1538              Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
1906                                                                               Trump arrested? Putin jailed? Fake AI images spread online   
2554    Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
...                                                                                                                                       ...   
196963  Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
198415  Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
198792            Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice   
198964  Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Sec

In [18]:
#this shows same news title was reported by different websites somwhere around few days or for the case of highest value_counts it is an act but 
# its content is different.
# this can also be interpreted as popularity of content  to drill deeper.

df_news_final_project[df_news_final_project['title']=='Trump arrested? Putin jailed? Fake AI images spread online']

url   
1906    https://www.wiltonbulletin.com/news/politics/article/trump-arrested-putin-jailed-fake-ai-images-17856545.php  \
10118               https://www.wmar2news.com/news/national/trump-arrested-putin-jailed-fake-ai-images-spread-online   
10829     https://www.ncadvertiser.com/news/politics/article/trump-arrested-putin-jailed-fake-ai-images-17856545.php   
12993                      https://www.waff.com/2023/03/23/trump-arrested-putin-jailed-fake-ai-images-spread-online/   
18830                   https://www.kgun9.com/news/national/trump-arrested-putin-jailed-fake-ai-images-spread-online   
...                                                                                                              ...   
190327  https://www.lakecountystar.com/news/politics/article/trump-arrested-putin-jailed-fake-ai-images-17856545.php   
192282                  https://www.fox10tv.com/2023/03/23/trump-arrested-putin-jailed-fake-ai-images-spread-online/   
196613                   https://www.wwnytv.com/2023/03/23/trump-arrested-putin-jailed-fake-ai-images-spread-online/   
196784                                        https://news.yahoo.com/trump-arrested-putin-jailed-fake-180305096.html   
199931                     https://www.kltv.com/2023/03/23/trump-arrested-putin-jailed-fake-ai-images-spread-online/   

             date language   
1906   2023-03-23       en  \
10118  2023-03-24       en   
10829  2023-03-23       en   
12993  2023-03-23       en   
18830  2023-03-24       en   
...           ...      ...   
190327 2023-03-23       en   
192282 2023-03-23       en   
196613 2023-03-23       en   
196784 2023-03-23       en   
199931 2023-03-23       en   

                                                             title   
1906    Trump arrested? Putin jailed? Fake AI images spread online  \
10118   Trump arrested? Putin jailed? Fake AI images spread online   
10829   Trump arrested? Putin jailed? Fake AI images spread online   
12993   Trump arrested? Putin jailed? Fake AI images spread online   
18830   Trump arrested? Putin jailed? Fake AI images spread online   
...                                                            ...   
190327  Trump arrested? Putin jailed? Fake AI images spread online   
192282  Trump arrested? Putin jailed? Fake AI images spread online   
196613  Trump arrested? Putin jailed? Fake AI images spread online   
196784  Trump arrested? Putin jailed? Fake AI images spread online   
199931  Trump arrested? Putin jailed? Fake AI images spread online   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

are text and title highest counts common articles? if not then just heading same but content different.??

In [19]:
value_counts = df_news_final_project['title'].value_counts()
top_counts = value_counts[value_counts > 1]

rows_with_counts = pd.DataFrame({'title': top_counts.index, 'count': top_counts.values})

rows_with_counts.head()

,title,count
0,Virtus Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice,237
1,Virtus AllianzGI Artificial Intelligence &amp; Technology Opportunities Fund Discloses Sources of Distribution - Section 19(a) Notice,119
2,Trump arrested? Putin jailed? Fake AI images spread online,99
3,"Musk, scientists call for halt to AI race sparked by ChatGPT",97
4,Biden administration seeks input on AI safety measures,86


In [20]:
text_counts = df_news_final_project['text'].value_counts()
top_counts = text_counts[text_counts > 1]

rows_with_counts_text = pd.DataFrame({'text': top_counts.index, 'count': top_counts.values})

rows_with_counts_text.head()

text   
0  \n\nSentinelOne Named to the 2020 CB Insights AI 100 List | | Consumer Electronics Net\n\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electronics Net\n\nSearch for:\n \nHomeNewsSentinelOne Named to the 2020 CB Insights AI 100 List \n \n                                 News\n                             \n \nSentinelOne Named to the 2020 CB Insights AI 100 List\n                    8 hours ago            \n \n\nOnly Endpoint Security Company Recognized as One of the Most Innovative Artificial Intelligence Startups\n\nMOUNTAIN VIEW, Calif.–(BUSINESS WIRE)–SentinelOne, the autonomous cybersecurity platform company, today announced the company was named to the fourth annual CB Insights AI 100 ranking, showcasing the 100 most promising private artificial intelligence companies in the world. SentinelOne is one of ten companies on this year’s list valued at over $1 billion and is the only endpoint security company included.\nSentinelOne is the only cybersecurity solution that encompasses prevention, detection, and response capabilities across endpoints, containers, cloud workloads, and IoT devices in a single, completely autonomous platform — using patented behavioral and static AI models to deliver autonomous capabilities with the lowest performance impact. With SentinelOne, organizations gain full transparency into everything that is happening across the network at machine speed — to defeat every attack, at every stage of the threat lifecycle — replacing traditional antivirus solutions.\n\n\n“Our use of AI and machine learning is revolutionizing enterprise security,” said Raj Rajamani, Chief Product Officer, SentinelOne. “We were one of the first products to introduce a pure AI based malware detection engine that doesn’t rely on a signature database. Our AI processes enormous amounts of data in real time to detect anomalous activity autonomously. Deployed in the world’s largest enterprises, SentinelOne brings unparalleled prevention and detection coupled with autonomous responses across all attack surfaces – made possible by our patented AI models.”\n\n\nThe CB Insights research team selected the 2020 AI 100 from nearly 5,000 companies based on several factors including patent activity, investor quality, news sentiment analysis, proprietary Mosaic scores, market potential, partnerships, competitive landscape, team strength, and tech novelty. The 2020 AI 100 companies span the globe, from the US, UK, China, Chile, and South Africa, and are supported by more than 600 investors.\n\n\n“It’s been remarkable to see the success of the companies named to the Artificial Intelligence 100 over the last four years. The 2019 AI 100 saw 48 companies go on to raise $4.9B of additional financing and nine got acquired,” said CB Insights CEO Anand Sanwal. “It has been gratifying to see that CB Insights’ data-driven approach to identifying the top AI companies using patents, customer traction, investor quality, market sizing and more has become so effective at picking the AI winners of tomorrow. We look forward to seeing what the 2020 AI 100 companies will accomplish over the course of this year and beyond.”\n\nTo learn more on how SentinelOne uses AI to detect and autonomously respond to malicious behavior immediately, offering prevention of attacks, detection, and most importantly machine speed responses such as on-device remediation and rollback, please visit: www.sentinelone.com.\n\n\nAbout SentinelOne\nSentinelOne is the only cybersecurity solution encompassing AI-powered prevention, detection, response and hunting across endpoints, containers, cloud workloads, and IoT devices in a single autonomous platform. With SentinelOne, organizations gain full transparency into everything happening across the network at machine speed – to defeat every attack, at every stage of the threat lifecycle. To learn more visit www.sentinelone.com or follow us at @SentinelOne, on LinkedIn or Facebook.\n\n\nAbout CB Insights

In [21]:
#same news covered by different channels, must be important (maybe internationally)
df_news_final_project[df_news_final_project['text']=='Clearview AI broke Canadian privacy law: watchdogs\n\n']

,url,date,language,title,text,website
9968,https://www.theifp.ca/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.theifp.ca/
30735,https://www.bramptonguardian.com/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.bramptonguardian.com/
61356,https://www.simcoe.com/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.simcoe.com/
81267,https://www.newhamburgindependent.ca/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.newhamburgindependent.ca/
123985,https://www.parrysound.com/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.parrysound.com/
135745,https://www.cambridgetimes.ca/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.cambridgetimes.ca/
177169,https://www.guelphmercury.com/news-story/10322836-clearview-ai-broke-canadian-privacy-law-watchdogs/,2021-02-03,en,Clearview AI broke Canadian privacy law: watchdogs,Clearview AI broke Canadian privacy law: watchdogs\n\n,https://www.guelphmercury.com/


In [22]:
df_news_final_project[['url','date','language','title','text','website']].nunique()

url         200332
date          1213
language         1
title       140223
text        198564
website       6601
dtype: int64

In [23]:
df_news_final_project.head(1)

url       date language   
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html 2021-03-18       en  \

                                                                                           title   
0  Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online  \

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

# Data Sampling

In [24]:
df_news_final_project.drop(columns = ['website', 'language'], inplace = True)
df_news_final_project.head(1)

url       date   
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html 2021-03-18  \

                                                                                           title   
0  Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online  \

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [25]:
# df_sample = df_news_final_project.sample(frac=0.01,random_state=41)
df_sample = df_news_final_project.copy()

In [26]:
df_sample.head(1)

url       date   
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html 2021-03-18  \

                                                                                           title   
0  Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online  \

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [27]:
len(df_news_final_project), len(df_sample)

(200332, 200332)

# Data Cleaning

- remove words with character-length greater than 11 (chose 11 cz some names are like 11 in length eg Jenna Colatruglio at index 179106, rduncanspringmarketinggroupcom, International Newswires)

In [28]:
# # Get a list of stopwords from the NLTK library
# stop = stopwords.words('english')

# # Don't remove these words. Thye are important for sentiment analysis.
# excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
#              'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
#              'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
#              'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
#              'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# # New stopword list
# stopwords = [word for word in stop if word not in excluding]


In [29]:
# len(stopwords)

In [30]:
# initialize lemmatizer
# lemmatizer = nltk.stem.WordNetLemmatizer()
#lemmatizer = nltk.WordNetLemmatizer()
def text_clean(text):
    # replace characters like |,\t,\r,@,#() with empty string
    cleaned_text = re.sub(r'[\|\t\n\r@#()]', '', text)
    # remove text between more than 1 continuous /n 
    #cleaned_text = re.sub(r'\n+\s*\n+\s*(\s*\w+\b\s*){0,}\s*\n\s*\n', " ", cleaned_text)
    # remove URLs
    cleaned_text = re.sub(r'http\S+', '', cleaned_text)
    # remove non word characters (emojis etc)
    cleaned_text = re.sub(r'[^\w\s,.]',' ',cleaned_text)
    # remove numbers and punctuation
    #cleaned_text = re.sub(r'[\d,.;:?!]+','',cleaned_text) #remove punctuation
    cleaned_text = re.sub(r'[0-9]', ' ', cleaned_text)
    # remove words with character-length greater than 11
    cleaned_text = re.sub(r'\b\w{12,}\b', "", cleaned_text)
    # Lower case
    #cleaned_text = cleaned_text.lower()
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+',' ', cleaned_text)
    #Tokenize
    #text_tokens = nltk.tokenize.word_tokenize(cleaned_text)
    # Remove single-character tokens
    #text_tokens = [token for token in text_tokens if len(token) > 1]
    # remove stopwords
    #cleaned_text = [word for word in text_tokens if not word in stopwords]
    # lemmatize words
    #cleaned_text = [lemmatizer.lemmatize(word) for word in cleaned_text]
    cleaned_text = ''.join([x if x in string.printable else '' for x in cleaned_text])
    return cleaned_text

In [31]:
%%time
df_sample['clean_title'] = df_sample['title'].parallel_apply(text_clean)

CPU times: user 264 ms, sys: 4.08 s, total: 4.35 s
Wall time: 4.48 s


In [32]:
%%time
df_sample['clean_text'] = df_sample['text'].parallel_apply(text_clean)

CPU times: user 3.51 s, sys: 9.46 s, total: 13 s
Wall time: 19.7 s


In [33]:
df_sample.drop(columns = ['text', 'title'], inplace = True)

In [34]:
df_sample.head(1)

url       date   
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html 2021-03-18  \

                                                                      clean_title   
0  Artificial improves parking efficiency in Chinese cities People s Daily Online  \

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [35]:
def tokenize(text, stopwords):
    # # remove text between more than 1 continuous /n 
    # text = re.sub(r'\n+\s*\n+\s*(\s*\w+\b\s*){0,}\s*\n\s*\n', " ", text)
    # remove non word characters (emojis etc)
    #text = re.sub(r'[^\w\s]',' ',text)
    # remove numbers and punctuation
    #cleaned_text = re.sub(r'[\d,.;:?!]+','',cleaned_text) #remove punctuation
    #cleaned_text = re.sub(r'[0-9]', ' ', cleaned_text)
    # Lower case
    #text = text.lower()
    # # Remove extra spaces
    # text = re.sub(r'\s+',' ', text)
    
    #Tokenize
    tokens = nltk.word_tokenize(text)
    
    
    # Remove single-character tokens
    tokens = [token for token in tokens if len(token) > 1]
    
    # Remove character tokens with length > 14
    #tokens = [token for token in tokens if len(token) <= 14]

    # Remove numbers
    #tokens = [token for token in tokens if not token.isnumeric()]
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]

    # Lowercase all tokens (default_stopwords are lowercase too)
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Remove stopwords
    # tokens = [token for token in tokens if token not in custom_stopwords]

#     #Lemmatize
#     wnl = nltk.WordNetLemmatizer()
#     tokens = [wnl.lemmatize(token) for token in tokens]
    
#     # Remove single-character tokens
#     tokens = [token for token in tokens if len(token) > 1]
    
    return tokens

In [36]:
def tokens_to_text(tokens):
    
    if  len(tokens)> 250:
        new_text = " ".join(tokens[0:250])
    else:
        new_text = " ".join(tokens)
    return new_text

In [37]:
%%time
df_sample['title_tokens'] = df_sample['clean_title'].parallel_apply(tokenize, stopwords = set(nlp.Defaults.stop_words))
df_sample['title_token_text'] = df_sample['title_tokens'].parallel_apply(tokens_to_text)

CPU times: user 1.34 s, sys: 9.5 s, total: 10.8 s
Wall time: 11.8 s


# Filtering
keeping articles related to ai/ml

In [38]:
df_sample[['title_token_text']].head(5)

,title_token_text
0,artificial improves parking efficiency chinese cities people daily online
1,children autism saw learning social skills boosted playing ai robot news parliament
2,forget ml ai industry focus february test rework solutions dataweek
3,strategy analytics smartphones sold globally ai powered consumer electronics net
4,olympus support endoscopic ai diagnosis education doctors india launch ai diagnostic support application business net


In [39]:
keywords = ['ai','metaverse','big data','chatgpt','alexa', 
            'data science','chatbot','data', 'generative ai', 'automated','meta','cloud','artificial intelligence',
            'open ai','openai','google','microsoft','imaging', 'machine learning','machine','prediction','ocr', 'analytics','natural language',
            'ai models', 'elon musk', 'ml', 'llms', 'llm', 'large language', 'image generation', 'stable diffusion',
            'dall-e', 'dalle', 'gpt', 'opensource', 'conversational ai', 'bot','algorithm', 'nlp','natural language processing',
            'computer vision', 'attention is all you need', 'attention','lamda', 'transformer', 'bert', 'nlu','natural language understanding',
            'prompt engineering', 'nlg', 'discriminiative models', 'fine tuning', 'generative pre trained transformer', 'hallucination',
            'generative','pretrained','transformer','transformers','multimodal', 'language model', 'n shot learning', 'nli', 
            'natural language generation', 'reinforcement learning', 'sequence modelling', 'tokenization', 'intent recognition', 
            'sentiment analysis', 'speech recognition', 'bard', 'automation', 'genai', 'revolution', 'gpu', 'data', 
            'python', 'api', 'github','copilot','bing','bart','deep learning','neural networks','gan','image','augmentation','digital'
           'large','neural','networks','supervised','unsupervised','reasoning''chatgpts']

keyw = [*set(keywords)]

In [40]:
def get_keywords(text):
    global keyw
    words = text
    total_count = 0
    for i in keyw:
        occurance_count = words.count(i)
        total_count += occurance_count
    return total_count

In [41]:
%%time
df_sample['keywords'] = df_sample['title_token_text'].parallel_apply(get_keywords)

CPU times: user 147 ms, sys: 4.7 s, total: 4.84 s
Wall time: 5.06 s


In [42]:
df_sample = df_sample[df_sample['keywords'] != 0]

In [43]:
df_sample.shape

(181646, 7)

In [44]:
df_sample.head(2)

url   
0                                                                                      http://en.people.cn/n3/2021/0318/c90000-9830122.html  \
1  http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/   

        date   
0 2021-03-18  \
1 2020-02-27   

                                                                                                          clean_title   
0                                      Artificial improves parking efficiency in Chinese cities People s Daily Online  \
1  Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot News Parliament   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [45]:
df_sample.drop(columns = ['keywords'], inplace = True)

In [46]:
# %%time
# df_sample['article_length'] = df_sample['clean_text'].parallel_apply(lambda x: len(x.split()))

In [47]:
# %%time
# df_sample['title_length'] = df_sample['clean_title'].parallel_apply(lambda x: len(x.split()))

In [48]:
# df_sample['article_length'].describe()

In [49]:
# plt.figure(figsize=(10,4))
# plt.boxplot(df_sample['article_length'], vert = 0)
# plt.show()

In [50]:
# plt.hist(df_sample['article_length'], color = "green", log=True)
# plt.grid()
# plt.show()

In [51]:
# df_sample[df_sample['article_length']<150].shape[0]

In [52]:
# df_sample[df_sample['article_length']>2500].shape[0]

In [53]:
# df_sample = df_sample[(df_sample['article_length']>150) & (df_sample['article_length']<2500)]

In [54]:
# df_sample.shape

In [55]:
# plt.figure(figsize=(10,4))
# plt.boxplot(df_sample['article_length'], vert = 0)
# plt.grid()
# plt.show()

In [56]:
# plt.hist(df_sample['article_length'])
# plt.show()

In [57]:
# df_sample.head(2)

In [58]:
%%time
df_sample['tokens'] = df_sample['clean_text'].parallel_apply(tokenize, stopwords = set(nlp.Defaults.stop_words))

CPU times: user 17.4 s, sys: 14.3 s, total: 31.6 s
Wall time: 58.9 s


In [59]:
%%time
df_sample['token_text'] = df_sample['tokens'].parallel_apply(tokens_to_text)

CPU times: user 31.3 s, sys: 20.6 s, total: 51.9 s
Wall time: 54.6 s


In [60]:
df_sample.columns

Index(['url', 'date', 'clean_title', 'clean_text', 'title_tokens',
       'title_token_text', 'tokens', 'token_text'],
      dtype='object')

In [61]:
# df_sample.drop(columns = ['article_length','title_length','title_tokens','title_token_text'], inplace = True)
df_sample.drop(columns = ['title_tokens','title_token_text'], inplace = True)

In [62]:
df_sample.head(1)

url       date   
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html 2021-03-18  \

                                                                      clean_title   
0  Artificial improves parking efficiency in Chinese cities People s Daily Online  \

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [63]:
df_sample.to_csv('./cleaned_tokenized_output_final.csv')